# Example Jupyter Notebook
In the following notebook, we will show how to build complex queries in GOR using the Python SDK to connect to our instance. First, as always, we load the gor magic extension to be able to use the `%gor` and `%%gor` syntax.
This notebook assumes you are familiar with the gor syntax.

In [1]:
# Install the python sdk, if needed
! pip install nextcode-sdk[jupyter] -U >/dev/null

In [2]:
# load the magic extension and imports
%reload_ext nextcode
import pandas as pd
%env LOG_QUERY=1

 Gor magic extension has been loaded. You can now use '%gor' and '%%gor' in this notebook
 * Python SDK Version: 0.2.10
 * Query API Version: 1.9.0
 * GOR Version: 9.4-SNAPSHOT (git SHA 8fc327bb08282d02f9e987e11c4073e64ec77677)
 * Root Endpoint: https://platform.wuxinextcodedev.com/api/query/
 * Current User: jonb@wuxinextcode.com
env: LOG_QUERY=1


#### Simple single-line gor expression

In [3]:
%gor nor #dbsnp# | top 2

Query 1953 returned 2 rows in 0.31 sec


Chrom    pos reference allele        rsids
0  chr1  10020        AA      A  rs775809821
1  chr1  10039         A      C  rs978760828

#### Single-line gor expression which stores results in local variable

In [4]:
var = %gor nor #dbsnp# | top 2

Query 1954 returned 2 rows in 0.21 sec


Chrom    pos reference allele        rsids
0  chr1  10020        AA      A  rs775809821
1  chr1  10039         A      C  rs978760828

#### Simple multi-line gor expression
Multi-line expressions are started with `%%`. Each statement should end with `;`

In [5]:
%%gor
  def #top# = 2;
  gor #dbsnp#
   | top #top#;

Query 1955 returned 2 rows in 0.21 sec


Chrom    pos reference allele        rsids
0  chr1  10020        AA      A  rs775809821
1  chr1  10039         A      C  rs978760828

#### Multi-line gor expression which stores results in local variable
To load results from a multi-line gor expression into a variable you can use the special `<<` operator

In [6]:
%%gor new_results <<
def #tops# = 10;
gor #dbsnp# | top #tops#;

Query 1956 returned 10 rows in 0.21 sec


In [7]:
new_results.index

RangeIndex(start=0, stop=10, step=1)

#### Store results in the user_data folder
Instead of fetching the data to the jupyter server you can also use the << operator to store files in the user_data folder within the project on NFS.

In [8]:
%%gor user_data/outfile.gorz <<
  pgor #dbsnp#
   | top 1000;

Query 1957 returned 38000 rows in 0.48 sec


In [9]:
# verify that the file is there by executing gorls
%gorls user_data outfile.gorz

outfile.gorz (345.1KiB)


In [10]:
# now you can gor this file and bring the results into a local dataframe
%gor user_data/outfile.gorz | GROUP chrom -count | top 2

Query 1959 returned 2 rows in 0.21 sec


Chrom  bpStart     bpStop  allCount
0   chr1        0  248956422      2000
1  chr10        0  133797422      2000

#### Gor expression which references a local variable
Local variables can be references using the `$xxx` syntax. This will expand the variable inline and is meant to be used for simple strings and numbers

In [11]:
n = 5

In [12]:
%%gor
  gor -p chr1:69000-70000 #dbsnp#
   | TOP $n;

Query 1960 returned 5 rows in 0.21 sec


Chrom    pos reference allele         rsids
0  chr1  69045         A      G  rs1360507296
1  chr1  69063         T      C  rs1260343719
2  chr1  69071         C      A  rs1314111151
3  chr1  69071         C      G  rs1314111151
4  chr1  69072         G      A  rs1379296286

#### Gor expression with defs and create statements

In [13]:
%%gor
def #number# = $n;
def #VEP_single# = UKBB/freezes/2019_0325/plink_wes/vep_single.gorz;
def #myfavouritefood# = pizza;

create mydbsnp = gor -p chr1:69000-70000 #dbsnp#
                  | TOP #number#;

gor [mydbsnp];

Query 1961 returned 5 rows in 0.21 sec


Chrom    pos reference allele         rsids
0  chr1  69045         A      G  rs1360507296
1  chr1  69063         T      C  rs1260343719
2  chr1  69071         C      A  rs1314111151
3  chr1  69071         C      G  rs1314111151
4  chr1  69072         G      A  rs1379296286

#### Compound expressions
Using the local variables you can construct compount expressions, allowing you to split up long gor scripts.

In [14]:
defs = """
def #myfirstdef# = 2;
def #myseconddef# = 2;

"""
creates = """
create mydbsnp = gor -p chr1:69000-70000 #dbsnp#
                  | TOP #myfirstdef#;
"""

In [15]:
%%gor
 $defs
 $creates

 gor [mydbsnp];

Query 1962 returned 2 rows in 0.21 sec


Chrom    pos reference allele         rsids
0  chr1  69045         A      G  rs1360507296
1  chr1  69063         T      C  rs1260343719

#### Virtual relations
You can load pandas dataframes into the query with the special `[var:xxx]` syntax.

In [16]:
allpns = pd.read_csv('data/allpns.tsv', delimiter='\t')

In [17]:
%%gor
  create bla = gor #dbsnp#
   | top 10;
  nor [var:allpns]
   | top 5;

Loading relations [var:allpns] from local state
Query 1963 returned 5 rows in 0.37 sec


PN
0  1000786
1  1629813
2  5137728
3  1001070
4  2489370

#### Cancelling queries
Try executing this long-running query and then click the stop button

In [19]:
results = %gor gor #dbsnp# | GROUP chrom -count

Query has been cancelled


### Using the SDK without magic syntax
The `%` and `%%` is simply syntactic sugar for executing queries through the python sdk. You can use the sdk directly as well and that is probably the preferred method for serious analysis.

#### Initializing the SDK
The SDK usually needs to be initialized with an API key and a project. However, in Wuxi notebooks this has already been done for you.

In [20]:
#%env GOR_API_KEY=[jwt token]
#%env GOR_API_PROJECT=[project name]
import nextcode
import pandas
# get a handle on a query service proxy
svc = nextcode.get_service("query")

#### Running basic queries

In [21]:
qry = svc.execute("gor #dbsnp# | top 2;")
print(f"Query {qry.query_id} is {qry.status}. Runtime was {qry.duration} ms and it produced {qry.line_count} rows.")
print(f"The query that was run: {qry.query}")
from pprint import pprint
df = qry.dataframe()
df

Query 1966 is DONE. Runtime was 42 ms and it produced 2 rows.
The query that was run: gor #dbsnp# | top 2;


Chrom    pos reference allele        rsids
0  chr1  10020        AA      A  rs775809821
1  chr1  10039         A      C  rs978760828

#### Analysing failed queries

In [22]:
qry = svc.execute("gor #unknowndef# | top 2;")
if qry.status == "FAILED":
    import sys
    print(f"Query failed with error: {qry.error['message']}\n", file=sys.stderr)
    print(f"Additional information available from qry.error[]: {list(qry.error.keys())}", file=sys.stderr)

Query failed with error: Resource not found for iterator: /mnt/csa/env/dev/projects/ukbb_hg38/#unknowndef# /mnt/csa/env/dev/projects/ukbb_hg38/#unknowndef#

Additional information available from qry.error[]: ['command', 'commandIndex', 'commandName', 'commandSource', 'description', 'errorType', 'gorMessage', 'message', 'requestId', 'stackTrace', 'type', 'uri']


#### Using virtual relations

In [23]:
import pandas as pd
import nextcode
svc = nextcode.get_service("query")

# use a dataframe as a virtual relation
df = pd.DataFrame(columns=["hello", "world"], data=[[1, 2]])
qry = svc.execute("gor [test]", test=df)
print(qry.dataframe())

# use a tsv string as a virtual relation
tsv = """hello\tworld\n1\t2"""
qry = svc.execute("gor [test]", test=tsv)
print(qry.dataframe())

# if your virtual relation uses special characters (:) in variable name
qry = svc.execute("gor [var:test]", relations=[{"name": "var:test", "data": df}])
print(qry.dataframe())

   hello  world
0      1      2
   hello  world
0      1      2
   hello  world
0      1      2
